In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
movie=pd.read_csv('C:\\Users\\LENOVO\\Documents\\MovieGenre.csv',encoding = 'latin1')

In [4]:
movie.head(6)

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
5,113277,http://www.imdb.com/title/tt113277,Heat (1995),8.2,Action|Crime|Drama,https://images-na.ssl-images-amazon.com/images...


In [5]:
movie.columns

Index(['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster'], dtype='object')

In [6]:
movie.drop(['imdbId', 'Imdb Link', 'IMDB Score', 'Poster'],axis=1,inplace=True)

In [7]:
movie.head(6)

,Title,Genre
0,Toy Story (1995),Animation|Adventure|Comedy
1,Jumanji (1995),Action|Adventure|Family
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy|Family|Romance
5,Heat (1995),Action|Crime|Drama


In [8]:
movie.shape

(40108, 2)

In [9]:
#we have to clean our title with regex inorder to remove extra character
def Clean_Title(Title):
    return re.sub("[^a-zA-Z0-9 ]","",Title)

In [10]:
movie["Clean_Title"]=movie["Title"].apply(Clean_Title)

In [11]:
movie.head(6)

,Title,Genre,Clean_Title
0,Toy Story (1995),Animation|Adventure|Comedy,Toy Story 1995
1,Jumanji (1995),Action|Adventure|Family,Jumanji 1995
2,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,Father of the Bride Part II (1995),Comedy|Family|Romance,Father of the Bride Part II 1995
5,Heat (1995),Action|Crime|Drama,Heat 1995


In [12]:
movie.drop(['Title'],axis=1,inplace=True)

In [13]:
movie.head(5)

,Genre,Clean_Title
0,Animation|Adventure|Comedy,Toy Story 1995
1,Action|Adventure|Family,Jumanji 1995
2,Comedy|Romance,Grumpier Old Men 1995
3,Comedy|Drama|Romance,Waiting to Exhale 1995
4,Comedy|Family|Romance,Father of the Bride Part II 1995


In [14]:
#build TFID Matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
vc=TfidfVectorizer(ngram_range=(1,2))
Tfid=vc.fit_transform(movie["Clean_Title"])

In [16]:
Tfid

<40108x113206 sparse matrix of type '<class 'numpy.float64'>'
	with 266415 stored elements in Compressed Sparse Row format>

In [18]:
#create a search function 
from sklearn.metrics.pairwise import cosine_similarity
#def search(Title):
Title= "Toy Story"
Title=Clean_Title(Title)
query_vc=vc.transform([Title])

# similarity between the search title
similarity=cosine_similarity(query_vc,Tfid).flatten()
indices=np.argpartition(similarity,-5)[-5:]
result=movie.iloc[indices]

In [19]:
result

,Genre,Clean_Title
25346,Animation|Short|Comedy,Toy Story Toons Small Fry 2011
22711,Animation|Short|Adventure,Toy Story of Terror 2013
0,Animation|Adventure|Comedy,Toy Story 1995
3028,Animation|Adventure|Comedy,Toy Story 2 1999
16036,Animation|Adventure|Comedy,Toy Story 3 2010


In [27]:
#create a search function 
from sklearn.metrics.pairwise import cosine_similarity
def search(Title):
    #Title= "War 2008"
    #Title=Clean_Title(Title)
    query_vc=vc.transform([Title])

    # similarity between the search title
    similarity=cosine_similarity(query_vc,Tfid).flatten()
    indices=np.argpartition(similarity,-5)[-5:]
    result=movie.iloc[indices]
    return result

In [28]:
result

,Genre,Clean_Title
25346,Animation|Short|Comedy,Toy Story Toons Small Fry 2011
22711,Animation|Short|Adventure,Toy Story of Terror 2013
0,Animation|Adventure|Comedy,Toy Story 1995
3028,Animation|Adventure|Comedy,Toy Story 2 1999
16036,Animation|Adventure|Comedy,Toy Story 3 2010


In [29]:
#Build Interactive Search Box
import ipywidgets as widgets
from IPython.display import display

In [30]:
movie_input=widgets.Text(
    value="Toy Story",
    description= "Movie Title:",
    disabled=False
)
# create an output widget
movie_list=widgets.Output()
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        Title =data["new"]
        if len(Title)>7:
            display(search(Title))
movie_input.observe(on_type, names='value') 

display(movie_input,movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [26]:
movie_input=widgets.Text(
    value="Toy Story",
    description= "Movie Title:",
    disabled=False
)
# create an output widget
movie_list=widgets.Output()
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        Title =data["new"]
        if len(Title)>7:
            display(search(Title))
movie_input.observe(on_type, names='value') 

display(movie_input,movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()